In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import re

In [3]:
indeed= pd.read_csv(r'E:/MS_DAEN/DAEN690/Datasets/indeed_chunk.csv')
indeed.head(3)

,jobid,apply_link,company_link,company_name,company_rating,company_reviews_count,country,country_code,current_url,date_posted,...,job_type,location,region,salary_formatted,srcname,input,timestamp,benefits,date_posted_parsed,qualifications
0,000779005ca85e59,https://www.indeed.com/applystart?jk=000779005...,https://www.indeed.com/cmp/Leidos?campaignid=v...,Leidos,3.7,1331.0,US,US,https://www.indeed.com/viewjob?jk=000779005ca8...,30+ days ago,...,"[""Full-time""]","Gaithersburg, MD 20879",NaN,NaN,careers.leidos.com,"{""job_id"":""000779005ca85e59"","" domain"":""https:...",2022-07-03T06:34:43.643Z,NaN,NaN,NaN
1,0009b65052c52899,https://www.indeed.com/applystart?jk=0009b6505...,https://www.indeed.com/cmp/New-York-Community-...,New York Community Bank,3.3,320.0,US,US,https://www.indeed.com/viewjob?jk=0009b65052c5...,26 days ago,...,NaN,"Commack, NY 11725",NaN,NaN,recruiting.adp.com,NaN,NaN,"[""Referral program""]",2022-05-14T14:11:20.068Z,NaN
2,000c66cdc2de4cc9,https://www.indeed.com/applystart?jk=000c66cdc...,https://www.indeed.com/cmp/Leidos?campaignid=v...,Leidos,3.7,1326.0,US,US,https://www.indeed.com/viewjob?jk=000c66cdc2de...,23 days ago,...,"[""Full-time""]","Pearl Harbor, HI 96860",NaN,NaN,careers.leidos.com,NaN,NaN,NaN,2022-05-17T14:12:09.432Z,NaN


In [4]:
indeed.columns

Index(['jobid', 'apply_link', 'company_link', 'company_name', 'company_rating',
       'company_reviews_count', 'country', 'country_code', 'current_url',
       'date_posted', 'description', 'description_text', 'domain', 'job_title',
       'job_type', 'location', 'region', 'salary_formatted', 'srcname',
       'input', 'timestamp', 'benefits', 'date_posted_parsed',
       'qualifications'],
      dtype='object')

In [5]:
indeed = indeed [['country','location','job_title','description_text','company_name']]

In [6]:
indeed

,country,location,job_title,description_text,company_name
0,US,"Gaithersburg, MD 20879",Sr. Endpoint Cyber Engineer,Description Job Description: The Leidos Cor...,Leidos
1,US,"Commack, NY 11725",Cyber Security SIEM Administrator,JOB SUMMARY \n The Information\Cyber Security ...,New York Community Bank
2,US,"Pearl Harbor, HI 96860",Cyber Intrusion Analyst,Description Job Description: Leidos has a c...,Leidos
3,US,"Brooklyn, NY 11201",Infrastructure Cyber Security Engineer - NG9-1-1,Company Overview Motorola Solutions is there ...,Motorola Solutions
4,US,"Miami Gardens, FL 33054",Associate Professor of Cybersecurity,A successful candidate will play an active rol...,Florida Memorial University
...,...,...,...,...,...
13119,US,"San Antonio, TX 78216",TruMLS Cyber Engineer,Basic Qualifications : \n Bachelor’s degree...,"General Dynamics Mission Systems, Inc"
13120,US,Melbourne VIC,Cyber Threat management,Work type: Permanent Full time\n \n Locat...,NAB - National Australia Bank
13121,US,"Fort Meade, MD 20755",COMPUTER SCIENTIST (CYBER),Duties \n Leads capability analysis/assess...,US Defense Information Systems Agency
13122,US,Österreich,Cyber Security Penetration Tester (m/w/div.),Unternehmensbeschreibung\n Bei Bosch gestalte...,ITK


In [7]:
indeed = indeed.drop_duplicates()

In [8]:
indeed_new = indeed[indeed.country == 'US']

In [9]:
indeed_new

,country,location,job_title,description_text,company_name
0,US,"Gaithersburg, MD 20879",Sr. Endpoint Cyber Engineer,Description Job Description: The Leidos Cor...,Leidos
1,US,"Commack, NY 11725",Cyber Security SIEM Administrator,JOB SUMMARY \n The Information\Cyber Security ...,New York Community Bank
2,US,"Pearl Harbor, HI 96860",Cyber Intrusion Analyst,Description Job Description: Leidos has a c...,Leidos
3,US,"Brooklyn, NY 11201",Infrastructure Cyber Security Engineer - NG9-1-1,Company Overview Motorola Solutions is there ...,Motorola Solutions
4,US,"Miami Gardens, FL 33054",Associate Professor of Cybersecurity,A successful candidate will play an active rol...,Florida Memorial University
...,...,...,...,...,...
13118,US,Wallonia,Cybersecurity Architect,What are my Responsibilities? Define cyber...,AKKA TECHNOLOGIES
13119,US,"San Antonio, TX 78216",TruMLS Cyber Engineer,Basic Qualifications : \n Bachelor’s degree...,"General Dynamics Mission Systems, Inc"
13120,US,Melbourne VIC,Cyber Threat management,Work type: Permanent Full time\n \n Locat...,NAB - National Australia Bank
13121,US,"Fort Meade, MD 20755",COMPUTER SCIENTIST (CYBER),Duties \n Leads capability analysis/assess...,US Defense Information Systems Agency


In [10]:
indeed_new[['City','State']] = indeed_new['location'].str.split(',',expand=True)

In [12]:
indeed_new['State'] = indeed_new['State'].str.strip()

In [13]:
#indeed_new[['State Abbr','Pin']] = indeed_new['State'].str.split(' ',1,expand = False)
indeed_new[['State Abbr','Pin']]=indeed_new['State'].str.rsplit(" ", n=1, expand=True)

In [14]:
indeed_new

,country,location,job_title,description_text,company_name,City,State,State Abbr,Pin
0,US,"Gaithersburg, MD 20879",Sr. Endpoint Cyber Engineer,Description Job Description: The Leidos Cor...,Leidos,Gaithersburg,MD 20879,MD,20879
1,US,"Commack, NY 11725",Cyber Security SIEM Administrator,JOB SUMMARY \n The Information\Cyber Security ...,New York Community Bank,Commack,NY 11725,NY,11725
2,US,"Pearl Harbor, HI 96860",Cyber Intrusion Analyst,Description Job Description: Leidos has a c...,Leidos,Pearl Harbor,HI 96860,HI,96860
3,US,"Brooklyn, NY 11201",Infrastructure Cyber Security Engineer - NG9-1-1,Company Overview Motorola Solutions is there ...,Motorola Solutions,Brooklyn,NY 11201,NY,11201
4,US,"Miami Gardens, FL 33054",Associate Professor of Cybersecurity,A successful candidate will play an active rol...,Florida Memorial University,Miami Gardens,FL 33054,FL,33054
...,...,...,...,...,...,...,...,...,...
13118,US,Wallonia,Cybersecurity Architect,What are my Responsibilities? Define cyber...,AKKA TECHNOLOGIES,Wallonia,None,None,None
13119,US,"San Antonio, TX 78216",TruMLS Cyber Engineer,Basic Qualifications : \n Bachelor’s degree...,"General Dynamics Mission Systems, Inc",San Antonio,TX 78216,TX,78216
13120,US,Melbourne VIC,Cyber Threat management,Work type: Permanent Full time\n \n Locat...,NAB - National Australia Bank,Melbourne VIC,None,None,None
13121,US,"Fort Meade, MD 20755",COMPUTER SCIENTIST (CYBER),Duties \n Leads capability analysis/assess...,US Defense Information Systems Agency,Fort Meade,MD 20755,MD,20755


In [15]:
indeed_new.to_csv(r'indeed.csv')

In [18]:
counties = pd.read_excel(r'E:/MS_DAEN/DAEN690/Datasets/us_cities_states_counties_mapping.xlsx')

In [21]:
indeed_new['City'] = indeed_new['City'].str.strip()
indeed_new['State Abbr'] = indeed_new['State Abbr'].str.strip()
counties['primary_city'] = counties['primary_city'].str.strip()
counties['state'] = counties['state'].str.strip()

In [23]:
indeed_county = pd.merge(indeed_new, counties, how='left', left_on=['State Abbr','City'], right_on=['state','primary_city'])

In [27]:
indeed_county

,country,location,job_title,description_text,company_name,City,State,State Abbr,Pin,primary_city,state,county
0,US,"Gaithersburg, MD 20879",Sr. Endpoint Cyber Engineer,Description Job Description: The Leidos Cor...,Leidos,Gaithersburg,MD 20879,MD,20879,Gaithersburg,MD,Montgomery County
1,US,"Commack, NY 11725",Cyber Security SIEM Administrator,JOB SUMMARY \n The Information\Cyber Security ...,New York Community Bank,Commack,NY 11725,NY,11725,Commack,NY,Suffolk County
2,US,"Pearl Harbor, HI 96860",Cyber Intrusion Analyst,Description Job Description: Leidos has a c...,Leidos,Pearl Harbor,HI 96860,HI,96860,NaN,NaN,NaN
3,US,"Brooklyn, NY 11201",Infrastructure Cyber Security Engineer - NG9-1-1,Company Overview Motorola Solutions is there ...,Motorola Solutions,Brooklyn,NY 11201,NY,11201,Brooklyn,NY,Kings County
4,US,"Miami Gardens, FL 33054",Associate Professor of Cybersecurity,A successful candidate will play an active rol...,Florida Memorial University,Miami Gardens,FL 33054,FL,33054,Miami Gardens,FL,Miami-Dade County
...,...,...,...,...,...,...,...,...,...,...,...,...
16423,US,"San Antonio, TX 78216",TruMLS Cyber Engineer,Basic Qualifications : \n Bachelor’s degree...,"General Dynamics Mission Systems, Inc",San Antonio,TX 78216,TX,78216,San Antonio,TX,Bexar County
16424,US,"San Antonio, TX 78216",TruMLS Cyber Engineer,Basic Qualifications : \n Bachelor’s degree...,"General Dynamics Mission Systems, Inc",San Antonio,TX 78216,TX,78216,San Antonio,TX,Comal County
16425,US,Melbourne VIC,Cyber Threat management,Work type: Permanent Full time\n \n Locat...,NAB - National Australia Bank,Melbourne VIC,None,None,None,NaN,NaN,NaN
16426,US,"Fort Meade, MD 20755",COMPUTER SCIENTIST (CYBER),Duties \n Leads capability analysis/assess...,US Defense Information Systems Agency,Fort Meade,MD 20755,MD,20755,Fort Meade,MD,Anne Arundel County


In [25]:
msa = pd.read_csv(r'E:/MS_DAEN/DAEN690/Datasets/msa_county.csv')

In [28]:
msa_indeed = pd.merge(msa,indeed_county, how='left', left_on=['County name (or Township name for the New England states)', 'State abbreviation'], right_on=['county','State Abbr'])

In [ ]:
msa_indeed.to_csv(r'msa_indeed.csv')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import nltk
import re


In [ ]:
def plot_wordcloud(text):
    nltk.download('stopwords')
    nltk.download('wordnet')
    stop=set(stopwords.words('english'))
    #stopwords.update(["drink", "now", "wine", "flavor", "flavors"])
    def _preprocess_text(text):
        corpus=[]
        stem=PorterStemmer()
        lem=WordNetLemmatizer()
        for news in text:
            #news = [w.lower() for w in news]
            words=[w for w in word_tokenize(str(news)) if (w not in stop)]
            words = [w.lower() for w in words]
            words=[lem.lemmatize(w) for w in words if len(w)>2]
            words = re.sub(r'[^a-zA-Z0-9\- ]', '', str(words))
            #print(words)
            corpus.append(words)
        return corpus
    
    corpus=_preprocess_text(text)
    
    #wordcloud = WordCloud(background_color='white',stopwords=set(STOPWORDS),#max_words=100,
        #max_font_size=30,scale=3,random_state=1)
    
    wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='white', colormap='Set2', collocations=False, stopwords = STOPWORDS)
    
    wordcloud=wordcloud.generate(str(corpus))
    wordcloud.to_file("E:/MS_DAEN/DAEN690/Datasets/indeed_allwords.png")

    fig = plt.figure(1, figsize=(15, 15))
    plt.axis("off")
    #plt.savefig(r'E:\MS_DAEN\DAEN690\Datasets\indeed_allwords.png', format="png")
    plt.imshow(wordcloud)
    plt.show()

In [ ]:
plot_wordcloud(indeed['description_text'])